In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOW_WIDTH = 480
WINDOW_HEIGHT = 640

In [3]:
# def sync_get_element_by_xpath(xpath):
#     wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
#     return driver.find_element_by_xpath(xpath)

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [5]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    
    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = driver.find_element_by_xpath("//a[contains(text(), 'BOOK THIS CLASS NOW')]")
    click_btn.click()

In [6]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [7]:
with open(r'/Users/huohsien/workspace/python/yoga_class_smart_booking/class_schedules/14 September - 20 September 2020.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [8]:
# course_schedule

In [9]:
book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=13)

courses_to_be_booked = []

c = helper.search_courses_by_date('17', course_schedule)
# c = helper.search_courses_by_name('Vinyasa Yoga', c)
c = helper.search_courses_by_teacher('Mei', c)

courses_to_be_booked.extend(c)

helper.list_courses_to_be_booked(courses_to_be_booked)

Date: 17 Sep Thu
-----------------------

Name: Core Yoga 核心瑜珈
Teacher: Mei Wang
Time: 11:00 - 12:00 PM


Name: Hatha Yoga 哈達瑜珈
Teacher: Mei Wang
Time: 09:30 - 10:30 AM


Name: Warm Stretch 溫和伸展
Teacher: Mei Wang
Time: 12:15 - 01:15 PM




In [10]:
# book_datetime = datetime.datetime(2020, 9, 13, 22, 0, 0)
# courses_to_be_booked = []

# c = helper.search_courses_by_date('17', course_schedule)
# c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# courses_to_be_booked.extend(c)

# c = helper.search_courses_by_date('17', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Patti', c)
# c = helper.search_courses_by_time('PM', c)
# courses_to_be_booked.extend(c)

# c = helper.search_courses_by_date('17', course_schedule)
# c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# courses_to_be_booked.extend(c)

# helper.list_courses_to_be_booked(courses_to_be_booked)

In [11]:
# signin_datetime = book_datetime - datetime.timedelta(seconds=600)

In [12]:
signin_datetime = book_datetime - datetime.timedelta(seconds=12)

In [13]:
def scheduled_sign_in(driver, wait):
    print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)
    
def reserve_class(driver, wait, course_to_be_booked):
    print("reserve_class({},{},{})".format(driver, wait, course_to_be_booked['time']))
    book_url = course_to_be_booked['href']
    driver.get(book_url)
    click_book_this_class_now(driver, wait)

In [14]:

drivers = []
waits = []

for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    scheduler = BackgroundScheduler()
    
    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * idx, 0);
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    scheduler.add_job(scheduled_sign_in, 'date', run_date= signin_datetime + datetime.timedelta(seconds=3 * idx), args=[driver, wait])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, course_to_be_booked])
    
    scheduler.start()
 


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


Run time of job "scheduled_sign_in (trigger: date[2020-09-14 00:46:21 CST], next run at: 2020-09-14 00:46:21 CST)" was missed by 0:00:01.526786
[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d81bff535811>", line 8, in <module>
    driver, wait = create_new_driver()
  File "<ipython-input-4-c213463a0c82>", line 14, in create_new_driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/selenium/webdriver/chrome/webdriver.py", line 73, in __init__
    self.service.start()
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/selenium/webdriver/common/service.py", line 102, in start
    time.sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/IPython/core/interactiveshell.p

TypeError: object of type 'NoneType' has no len()